In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

dataset = pd.read_csv("PromptDataset.csv") # paste the path in these quotes
dataset.head()

,Prompt,Response
0,What is the capital of France?,The capital of France is Paris.
1,Explain the water cycle.,The water cycle is the continuous movement of ...
2,Name the three branches of the U.S. government...,The three branches of the U.S. government are ...
3,Describe how photosynthesis works.,Photosynthesis is the process in which green p...
4,What are the main differences between classica...,The main differences between classical and qua...


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split

# 80% training, 10% validation, and 10% test
train_size = 0.8
# we will use the val for adjusting the hyperparameters
val_size = 0.1
test_size = 0.1

train_df, temp_df = train_test_split(dataset, train_size=train_size, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=test_size / (val_size + test_size), random_state=42)

train_df.to_csv('PromptDataset_train.csv', index=False)
val_df.to_csv('PromptDataset_val.csv', index=False)
test_df.to_csv('PromptDataset_test.csv', index=False)

print("Train/validation/test split completed. Files saved as 'PromptDataset_train.csv', 'PromptDataset_val.csv', and 'PromptDataset_test.csv'.")

Train/validation/test split completed. Files saved as 'PromptDataset_train.csv', 'PromptDataset_val.csv', and 'PromptDataset_test.csv'.


In [ ]:
# Load the validation DataFrame
val_df = pd.read_csv('PromptDataset_val.csv')

# Print column names to verify
print("Columns in the DataFrame:", val_df.columns)

# Check the first few rows to confirm correct data loading
print(val_df.head())
reference_answer = row


Columns in the DataFrame: Index(['Prompt', 'Response'], dtype='object')
                                              Prompt  \
0  Discuss the concpt of free will versus determi...   
1  Discuss the impact of social media on mental h...   
2  Describe the cultural impact of the Beatles on...   
3  How can you prioritize tasks when you have mul...   
4  How do internships benefit students in their c...   

                                            Response  
0  The debate between free will and determinism c...  
1  Social media has a huge impact on mental healt...  
2  The Beatles had a profound cultural impact on ...  
3  I prioritize tasks when I have multiple deadli...  
4  Internships benefit students in their career d...  


NameError: name 'row' is not defined

In [ ]:
import openai
from openai import OpenAI
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Load and split dataset into validation and test sets
def split_dataset(dataset, validation_size=0.5, test_size=0.5):
    assert (validation_size + test_size) == 1.0, "Sizes must sum to 1"
    validation_data, test_data = train_test_split(dataset, test_size=test_size)
    return validation_data, test_data

# Define your dataset
dataset = pd.read_csv("PromptDataset.csv")

# Split dataset
validation_data, test_data = split_dataset(dataset)

# Define hyperparameter values to test
n_epochs_values = [10, 20, 30]
batch_size_values = [4, 8, 16]
learning_rate_values = [0.1, 0.3, 0.5]

client = OpenAI(
    api_key= 'key
openai.api_key = 'key'

# Function to generate text
def generate_text(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating text: {e}")
        return ""

# Function to evaluate hyperparameters using cosine similarity on validation data
def evaluate_hyperparameters(n_epochs, batch_size, learning_rate, val_df):
    try:
        vectorizer = TfidfVectorizer()
        reference_answers = val_df['Response'].tolist()
        vectorizer.fit(reference_answers)

        max_score = -np.inf  # Initialize to negative infinity

        for index, row in val_df.iterrows():
            prompt = row['Prompt']
            reference_answer = row['Response']
            generated_text = generate_text(prompt)

            # Vectorize the generated text and the reference answer
            generated_vector = vectorizer.transform([generated_text])
            reference_vector = vectorizer.transform([reference_answer])

            # Compute cosine similarity between the generated text and reference answer
            cosine_similarity_score = cosine_similarity(generated_vector, reference_vector)[0][0]

            # Track the highest similarity score
            if cosine_similarity_score > max_score:
                max_score = cosine_similarity_score

        return max_score

    except Exception as e:
        print(f"Error in evaluation: {e}")
        return 0

# Grid search for hyperparameters using validation data only
best_hyperparams = None
best_score = float('-inf')

for n_epochs, batch_size, learning_rate in itertools.product(n_epochs_values, batch_size_values, learning_rate_values):
    print(f"Testing: n_epochs={n_epochs}, batch_size={batch_size}, learning_rate={learning_rate}")

    score = evaluate_hyperparameters(n_epochs, batch_size, learning_rate, validation_data)

    if score > best_score:
        best_score = score
        best_hyperparams = {
            "n_epochs": n_epochs,
            "batch_size": batch_size,
            "learning_rate_multiplier": learning_rate
        }

print(f"Best hyperparameters: {best_hyperparams}")
print(f"Best validation score: {best_score}")


Testing: n_epochs=10, batch_size=4, learning_rate=0.1
Testing: n_epochs=10, batch_size=4, learning_rate=0.3
Testing: n_epochs=10, batch_size=4, learning_rate=0.5
Testing: n_epochs=10, batch_size=8, learning_rate=0.1
Testing: n_epochs=10, batch_size=8, learning_rate=0.3
Testing: n_epochs=10, batch_size=8, learning_rate=0.5
Testing: n_epochs=10, batch_size=16, learning_rate=0.1
Testing: n_epochs=10, batch_size=16, learning_rate=0.3
Testing: n_epochs=10, batch_size=16, learning_rate=0.5
Testing: n_epochs=20, batch_size=4, learning_rate=0.1
Testing: n_epochs=20, batch_size=4, learning_rate=0.3
Testing: n_epochs=20, batch_size=4, learning_rate=0.5
Testing: n_epochs=20, batch_size=8, learning_rate=0.1
Testing: n_epochs=20, batch_size=8, learning_rate=0.3
Testing: n_epochs=20, batch_size=8, learning_rate=0.5
Testing: n_epochs=20, batch_size=16, learning_rate=0.1
Testing: n_epochs=20, batch_size=16, learning_rate=0.3
Testing: n_epochs=20, batch_size=16, learning_rate=0.5
Testing: n_epochs=30, 

In [ ]:
import openai
from openai import OpenAI
import numpy as np
import pandas as pd
import itertools

# Split dataset into training, validation, and test sets
def split_dataset(dataset, train_size=0.8, validation_size=0.1, test_size=0.1):
    assert (train_size + validation_size + test_size) == 1.0, "Sizes must sum to 1"

    train_data, temp_data = train_test_split(dataset, train_size=train_size)
    validation_data, test_data = train_test_split(temp_data, test_size=test_size / (validation_size + test_size))

    return train_data, validation_data, test_data

# Define your dataset
dataset = "PromptDataset.csv"

# Split dataset
train_data, validation_data, test_data = split_dataset(dataset)

# Define hyperparameter values to test
n_epochs_values = [10, 20, 30]
batch_size_values = [4, 8, 16]
learning_rate_values = [0.1, 0.3, 0.5]

client = OpenAI(
    api_key= 'key',
)
openai.api_key = 'key'

def submit_finetuning_job(n_epochs, batch_size, learning_rate_multiplier, train_data, validation_data):
    response = client.fine_tuning.jobs.create(
        training_file=training_file_id,
        validation_file=validation_file_id,

        model="gpt-3.5-turbo",
        suffix="recipe-ner",
        hyperparameters={
            "n_epochs": n_epochs,
            "batch_size": batch_size,
            "learning_rate_multiplier": learning_rate_multiplier
        }
    )
    return response.id

# Function to monitor job status
def monitor_job(job_id):
    while True:
        response = client.fine_tuning.jobs.retrieve(job_id)
        status = response.status
        print(f"Job ID: {job_id}, Status: {status}")
        if status in ["succeeded", "failed"]:
            return status
        time.sleep(60)  # Wait for 1 minute before checking again

# Function to evaluate hyperparameters
def evaluate_hyperparameters(params):
    try:
        prompts = validation_data['Prompt'].tolist()  # Use validation data
        total_score = 0
        for prompt in prompts:
            generated_text = generate_text(prompt, params['max_tokens'], params['temperature'])
            quality_score = len(generated_text) / params['max_tokens']
            total_score += quality_score
        average_score = total_score / len(prompts)
        return average_score
    except Exception as e:
        print(f"Error in evaluation: {e}")
        return 0

# Function to generate text (placeholder; replace with actual text generation logic)
def generate_text(prompt, max_tokens, temperature):
    # Placeholder function for generating text
    return "Generated text based on prompt"

# Grid search for hyperparameters
best_hyperparams = None
best_score = float('-inf')

for n_epochs, batch_size, learning_rate in itertools.product(n_epochs_values, batch_size_values, learning_rate_values):
    print(f"Testing: n_epochs={n_epochs}, batch_size={batch_size}, learning_rate={learning_rate}")

    job_id = submit_finetuning_job(n_epochs, batch_size, learning_rate, train_data, validation_data)
    status = monitor_job(job_id)

    if status == "succeeded":
        # Evaluate the job
        params = {
            'max_tokens': 50,  # Example parameter, adjust as needed
            'temperature': 0.7  # Example parameter, adjust as needed
        }
        score = evaluate_hyperparameters(params)

        if score > best_score:
            best_score = score
            best_hyperparams = {
                "n_epochs": n_epochs,
                "batch_size": batch_size,
                "learning_rate_multiplier": learning_rate
            }

print(f"Best hyperparameters: {best_hyperparams}")
print(f"Best validation score: {best_score}")

# Train final model on combined training and validation data with the best hyperparameters
final_model_id = submit_finetuning_job(
    n_epochs=best_hyperparams["n_epochs"],
    batch_size=best_hyperparams["batch_size"],
    learning_rate_multiplier=best_hyperparams["learning_rate_multiplier"],
    train_data=train_data + validation_data,
    validation_data=None  # No validation data here
)

# Monitor final model training
final_model_status = monitor_job(final_model_id)
if final_model_status == "succeeded":
    # Evaluate the final model on test data
    final_score = evaluate_hyperparameters(params)  # Adjust as needed for final evaluation
    print(f"Final test score: {final_score}")


Testing: n_epochs=10, batch_size=4, learning_rate=0.1
Job ID: ftjob-yBfv7Phin9UASDpRBdGbdGTn, Status: validating_files


KeyboardInterrupt: 

In [ ]:
import openai
from openai import OpenAI
import numpy as np
import pandas as pd
import itertools
import time
from sklearn.model_selection import train_test_split

# Initialize OpenAI client
client = OpenAI(
    api_key='key',
)

# Function to upload files and get their IDs
def upload_file(file_path):
    with open(file_path, 'rb') as file:
        response = openai.File.create(
            file=file,
            purpose='fine-tune'
        )
    return response['id']

# Upload your training and validation files
training_file_id = upload_file('PromptDataset_train.csv')
validation_file_id = upload_file('PromptDataset_val.csv')

# Split dataset into training, validation, and test sets
def split_dataset(dataset, train_size=0.8, validation_size=0.1, test_size=0.1):
    assert (train_size + validation_size + test_size) == 1.0, "Sizes must sum to 1"

    train_data, temp_data = train_test_split(dataset, train_size=train_size)
    validation_data, test_data = train_test_split(temp_data, test_size=test_size / (validation_size + test_size))

    return train_data, validation_data, test_data

# Load dataset
dataset = pd.read_csv("PromptDataset.csv")

# Split dataset
train_data, validation_data, test_data = split_dataset(dataset)

# Define hyperparameter values to test
n_epochs_values = [10, 20, 30]
batch_size_values = [4, 8, 16]
learning_rate_values = [0.1, 0.3, 0.5]

def submit_finetuning_job(n_epochs, batch_size, learning_rate_multiplier):
    response = client.fine_tuning.jobs.create(
        training_file=training_file_id,
        validation_file=validation_file_id,
        model="gpt-3.5-turbo",
        suffix="recipe-ner",
        hyperparameters={
            "n_epochs": n_epochs,
            "batch_size": batch_size,
            "learning_rate_multiplier": learning_rate_multiplier
        }
    )
    return response['id']

# Function to monitor job status
def monitor_job(job_id):
    while True:
        response = client.fine_tuning.jobs.retrieve(job_id)
        status = response['status']
        print(f"Job ID: {job_id}, Status: {status}")
        if status in ["succeeded", "failed"]:
            return status
        time.sleep(60)  # Wait for 1 minute before checking again

# Function to evaluate hyperparameters
def evaluate_hyperparameters(params):
    try:
        prompts = validation_data['Prompt'].tolist()  # Use validation data
        total_score = 0
        for prompt in prompts:
            generated_text = generate_text(prompt, params['max_tokens'], params['temperature'])
            quality_score = len(generated_text) / params['max_tokens']
            total_score += quality_score
        average_score = total_score / len(prompts)
        return average_score
    except Exception as e:
        print(f"Error in evaluation: {e}")
        return 0

# Placeholder function for generating text
def generate_text(prompt, max_tokens, temperature):
    # Use the actual OpenAI text generation logic here
    return "Generated text based on prompt"

# Grid search for hyperparameters
best_hyperparams = None
best_score = float('-inf')

for n_epochs, batch_size, learning_rate in itertools.product(n_epochs_values, batch_size_values, learning_rate_values):
    print(f"Testing: n_epochs={n_epochs}, batch_size={batch_size}, learning_rate={learning_rate}")

    job_id = submit_finetuning_job(n_epochs, batch_size, learning_rate)
    status = monitor_job(job_id)

    if status == "succeeded":
        # Evaluate the job
        params = {
            'max_tokens': 50,  # Example parameter, adjust as needed
            'temperature': 0.7  # Example parameter, adjust as needed
        }
        score = evaluate_hyperparameters(params)

        if score > best_score:
            best_score = score
            best_hyperparams = {
                "n_epochs": n_epochs,
                "batch_size": batch_size,
                "learning_rate_multiplier": learning_rate
            }

print(f"Best hyperparameters: {best_hyperparams}")
print(f"Best validation score: {best_score}")

# Train final model on combined training and validation data with the best hyperparameters
final_model_id = submit_finetuning_job(
    n_epochs=best_hyperparams["n_epochs"],
    batch_size=best_hyperparams["batch_size"],
    learning_rate_multiplier=best_hyperparams["learning_rate_multiplier"]
)

# Monitor final model training
final_model_status = monitor_job(final_model_id)
if final_model_status == "succeeded":
    # Evaluate the final model on test data
    final_score = evaluate_hyperparameters(params)  # Adjust as needed for final evaluation
    print(f"Final test score: {final_score}")


In [ ]:
import json

def csv_to_jsonl(csv_file_path, jsonl_file_path):
    # Load the CSV file into a DataFrame
    dataset = pd.read_csv(csv_file_path)

    # Open the JSONL file for writing
    with open(jsonl_file_path, 'w') as jsonl_file:
        for _, row in dataset.iterrows():
            # Construct the JSON object for each row
            json_obj = {
                "messages": [
                    {"role": "user", "content": row['Prompt']},
                    {"role": "assistant", "content": row['Response']}
                ]
            }
            # Write the JSON object to the JSONL file
            jsonl_file.write(json.dumps(json_obj) + '\n')

    print(f"CSV file '{csv_file_path}' converted to JSONL format and saved as '{jsonl_file_path}'.")


In [ ]:
# Paths for the CSV and JSONL files
csv_files = [
    ("PromptDataset_train.csv", "PromptDataset_train.jsonl"),
    ("PromptDataset_test.csv", "PromptDataset_test.jsonl")
]

# Convert each CSV file to JSONL format
for csv_file_path, jsonl_file_path in csv_files:
    csv_to_jsonl(csv_file_path, jsonl_file_path)


CSV file 'PromptDataset_train.csv' converted to JSONL format and saved as 'PromptDataset_train.jsonl'.
CSV file 'PromptDataset_test.csv' converted to JSONL format and saved as 'PromptDataset_test.jsonl'.


In [ ]:
!head -n 5 PromptDataset_train.jsonl

{"messages": [{"role": "user", "content": "What are the seven continents of the world?"}, {"role": "assistant", "content": "The seven continents of the world are Asia, Africa, North America, South America, Europe, and Australia.\r"}]}
{"messages": [{"role": "user", "content": "What are the main differences between classical and quantum computing?"}, {"role": "assistant", "content": "The main differences between classical and quantum computing are the methods and speed of each type of computing. Classical computing uses bits to represent data for binary processing and quantum computing uses quantum properties of atomic and subatomic particles. Classical computing uses bits as the basic unit of information, which can be either 0 or 1, and linearly performs operations with stable and well-established error correction methods.  Quantum computing uses qubits, which can exist in a superposition of states, allowing for simultaneous calculations and solving specific problems more efficiently u

In [ ]:
!head -n 5 PromptDataset_test.jsonl

{"messages": [{"role": "user", "content": "Explain the philosophy of utilitarianism."}, {"role": "assistant", "content": "Utilitarianism is a consequentialist ethical theory that posits that the rightness or wrongness of actions is determined by their outcomes. People high in openness might appreciate utilitarianism\u2019s innovative and flexible approach to morality, emphasizing the greatest happiness principle, which states that actions are morally right if they promote the greatest amount of happiness or pleasure for the greatest number of people. Those with high conscientiousness might be drawn to the theory\u2019s structured method of evaluating actions based on their consequences. Utilitarianism requires an impartial consideration of the interests of all affected parties, which aligns with the agreeableness trait, as it involves empathy and concern for others' well-being. Extraverts might find the social aspect of maximizing overall happiness appealing, as it often involves colla

In [ ]:
import openai
from openai import OpenAI


client = OpenAI(
    api_key= 'key',
)
openai.api_key = 'key'

def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id

training_file_id = upload_file("PromptDataset_train.jsonl", "fine-tune")
validation_file_id = upload_file("PromptDataset_test.jsonl", "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-KBuDvg8q920PAr8GDtdpE6UN
Validation file ID: file-w0C6ZsUCtm5bMbPx3qz2gLuW


In [ ]:
MODEL = "gpt-3.5-turbo"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="recipe-ner",
    hyperparameters={
      'n_epochs': 10,
      'batch_size': 4,
      'learning_rate_multiplier': 0.3
  },
)



job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-gTkpVIJ0AcHCo3z6EYdf3DFz
Status: validating_files


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-gTkpVIJ0AcHCo3z6EYdf3DFz
Status: validating_files
Trained Tokens: None


In [ ]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-gTkpVIJ0AcHCo3z6EYdf3DFz
Validating training file: file-KBuDvg8q920PAr8GDtdpE6UN and validation file: file-w0C6ZsUCtm5bMbPx3qz2gLuW


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:behavioralllm:recipe-ner:9urwnEjj


In [ ]:
test_messages = []
test_messages.append({"role": "user", "content": "What are the key milestones in the development of renewable energy technologies?"})

In [ ]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, max_tokens = 500, temperature = 0.7,
)
print(response.choices[0].message.content)

The key milestones in the development of renewable energy technologies include:

1. Late 19th century: The invention of the first solar cell by Charles Fritts in 1883.

2. 1970s: The rise of wind power with the construction of the first commercial wind turbines.

3. 1980s: The introduction of the first grid-connected photovoltaic power systems.

4. 1990s: The development of biopower plants using biomass as a fuel source.

5. Early 2000s: The commercialization of wave and tidal power technologies.

6. 2010s: The rapid growth of solar and wind power installations worldwide.

7. Today: The ongoing research and development of renewable energy technologies to increase efficiency and reduce costs.
